# Justice League Stellar Merger History

## Charlotte Christensen, June 11 2025

This jupyter notebook runs Anna Wright's code (/home/christenc/Code/python/AnnaWrite_startrace/RomZoomSHAnalysisScripts) to identify the stars in those halos.

From Anna's Email (June 4, 2025)

I managed to get through the pipeline and remember what everything did well enough to comment it today, but didn't have time to test it on a new halo. However, if you'd like to try it in the next 24 hours or so, I've attached a zip file with steps 1-7 of my pipeline (there are 8 files, but TrackDownStars_rz.ipynb and FixHostIDs_rz.py are really two halves of a single step). Step 0 is creating a tangos db for the simulation you'll be working with and my pipeline uses that and the simulation itself to create an hdf5 file with relevant data for each star particle. The most important bits are a unique host ID for each star particle so that stars that formed in the same halo can be grouped together, even if that halo doesn't exist at z=0, and the orbital circularity of each star particle, which I use to identify members of the stellar halo.
I will be testing it tomorrow on one of the newer Romulus Zooms, so there's a good chance I'll be sending you an updated version very soon with any bug fixes :) 
I apologize for how many pieces the pipeline is in - it really isn't all that complicated, but it was adapted from what I did for the FOGGIE sims and I split the steps of that pipeline up so that I could do as much as possible locally (rather than on Pleiades) and so that I could sanity check as often as possible. Please let me know if you have any issues, questions, or suggestions! I'd definitely be eager to hear what Juan does differently!

In [1]:
import os
os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'
os.chdir('/home/selvani/MAP/pynbody/AnnaWright_startrace/')

import tangos
tangos.all_simulations()

[<Simulation("cptmarvel.4096g5HbwK1BH_bn")>]

In [2]:
import pynbody as pb
import numpy as np
import pandas as pd
import glob
import h5py
import tangos

# For importing modules
import importlib.util
import sys
from pathlib import Path

In [3]:
# Simulation name and path

# Elena
# simname = 'h329' # not needed?
# res = '100'  # The Near Mint runs not needed?
#res = 'Mint'  # The Mint
simpath = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
basename = 'cptmarvel.cosmo25cmb.4096g5HbwK1BH'

ss_dir = 'cptmarvel.4096g5HbwK1BH_bn'#'snapshots_200crit_h329' # same as db_sim?
sim_base = simpath + ss_dir + '/'
ss_z0 = sim_base + basename + '.004096'

outfile_dir = "/home/selvani/MAP/pynbody/stellarhalo_trace_aw/"

### 1) GrabTF_rz.py

Step 1 of stellar halo pipeline

**What it does:**
- Loads the final snapshot (z=0) of the simulation
- Extracts formation time (`tform`) and particle IDs (`iord`) for all star particles that have `tform > 0` (i.e., actual stars, not wind particles)
- Converts formation times to Gyr units
- Creates a 2D array with particle IDs in the first row and formation times in the second row
- Saves this data as a `.npy` file named `<simulation_name>_tf.npy`
- Prints the total number of star particles found as a sanity check

**Input:** Simulation snapshot (specifically the final snapshot `.004096`)

**Output:** `<sim>_tf.npy` - NumPy file containing:
- Row 0: Star particle IDs (`iord`)  
- Row 1: Formation times in Gyr (`tform`)

**Purpose:** This creates the foundation dataset that subsequent steps will use to trace back each star particle to determine which halo it was forming in at its birth time.

* Usage:   `python GrabTF_rz.py <simpath> <output_directory>`
* Example: `python GrabTF_rz.py /path/to/sim/ /path/to/output/`

In [26]:
# Import Anna's code, even if not along my path
file_path = '/home/selvani/MAP/pynbody/AnnaWright_startrace/RomZoomSHAnalysisScripts/GrabTF_rz.py'
module_name = 'GrabTF_rz'

spec = importlib.util.spec_from_file_location(module_name, file_path)
module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(module)

In [27]:
module.main(simpath, outfile_dir)

414596 stars found!
Save to  /home/selvani/MAP/pynbody/stellarhalo_trace_aw/cptmarvel.cosmo25cmb.4096g5HbwK1BH_tf.npy


### 2) LocAtCreation_pool_rz.py

<!-- Step 2 of stellar halo pipeline -->
Identifies the host of each star particle in \<sim\>_tf.npy at the 
time it was formed. **Note that what is stored is NOT the amiga.grp 
ID, but the index of that halo in the tangos database. The amiga.grp
ID can be backed out via tangos with sim[stepnum][halonum].finder_id.** 
(CC, I believe that I edited this so now the halo_)

<!-- Output: <sim>_stardata_<snapshot>.h5
        where <snapshot> is the first snapshot that a given process
        analyzed. There will be <nproc> of these files generated
        and processes will not necessarily analyze adjacent snapshots -->

<!-- * Usage:   python LocAtCreation_pool_rz.py <sim> optional:<nproc>
* Example: python LocAtCreation_pool_rz.py r634 2 -->

<!-- Includes an optional argument to specify number of processes to run
with; default is 4. Note that this will get reduced if you've specified
more processes than you have snapshots to process. -->

<!-- Note that this has the name of the snapshots directory hardcoded into FindHaloStars.py (L63)
-- Will need to be adjusted
The 

CC: When I did my edits, I moved code into FindHaloStars so that it can be imported for multiprocessing -->


Step 2 of stellar halo pipeline

**What it does:**
- Loads the `<sim>_tf.npy` file created in step 1 (containing star particle IDs and formation times)
- Queries the Tangos database to get all available simulation snapshots and their cosmic times
- Determines which snapshots contain newly formed stars by binning star formation times
- Uses multiprocessing to analyze snapshots in parallel for efficiency
- For each relevant snapshot, identifies which halo each star particle belonged to at the time it formed
- Extracts additional data for each star: formation position, formation time, and host halo ID
- Converts Amiga halo group IDs to Tangos database indices for consistency
- Handles unbound particles (those not in any halo) by assigning them host ID = -1

**Detailed Process:**
1. **Data Loading**: Loads the `_tf.npy` file containing star particle IDs and formation times
2. **Snapshot Analysis**: Gets all simulation timesteps from Tangos database and sorts by cosmic time
3. **Star Distribution**: Creates histogram of star formation times to identify which snapshots contain new stars
4. **Chunk Creation**: Divides snapshots among multiple processes for parallel processing
5. **Multiprocessing Execution**: Each process handles a subset of snapshots independently

**FindHaloStars Function (called by each process):**
- **Time Matching**: For each snapshot, finds stars that formed between the previous snapshot and current one
- **Particle Matching**: Uses `iord` (particle IDs) to match stars from step 1 with their counterparts in historical snapshots
- **Host Identification**: Determines which halo (`amiga.grp`) each star was in when it formed
- **Database Indexing**: Converts halo IDs to Tangos database indices using a lookup dictionary
- **Position/Time Extraction**: Records formation positions, times, and snapshot locations
- **Data Writing**: Periodically saves data to HDF5 files to manage memory usage

**Key Technical Details:**
<!-- - Each process loads the same `_tf.npy` data independently to avoid sharing conflicts -->
- Uses `np.searchsorted()` for efficient particle ID matching between snapshots
- Creates `fid` dictionary to map Amiga group IDs to Tangos database indices
- Handles missing particles gracefully (assigns host ID = -1 for unbound stars)

**Input:** 
- `<sim>_tf.npy` from step 1
- Simulation snapshots (all timesteps)
- Tangos database connection

**Output:** `<sim>_stardata_<snapshot>.h5` files (one per process) containing:
- `particle_IDs`: Star particle IDs (`iord`)
- `particle_positions`: 3D positions at formation time (Mpc)
- `particle_creation_times`: Formation times (Gyr)
- `timestep_location`: Snapshot number where star was first found
- `particle_hosts`: Host halo index in Tangos database (-1 for unbound stars)

<!-- **Performance Features:**
- **Multiprocessing**: Uses all available CPU cores (up to 72 logical cores) for parallel processing
- **Load Balancing**: Randomly shuffles snapshot order to distribute work evenly
- **Memory Management**: Periodically writes data to disk to prevent memory overflow
- **Process Isolation**: Each process works independently to avoid conflicts -->

**Important Notes:**
- **Host IDs are Tangos database indices, NOT Amiga group IDs**
- Multiple output files are created (one per process) that will be combined in later steps
<!-- - Uses multiprocessing for significant speed improvement on multi-core systems -->
<!-- - Automatically handles load balancing by shuffling snapshot order -->
- Each process creates its own output file named by the first snapshot it processes

* Usage: `python LocAtCreation_pool_rz.py <simpath> <db_sim_name> <output_dir> [n_processes]`
* Example: `python LocAtCreation_pool_rz.py /path/to/sim/ cptmarvel.4096g5HbwK1BH_bn /output/ 36`

**Purpose:** This step creates the detailed formation history for each star particle, linking it to its birth halo and enabling stellar halo analysis. The multiprocessing approach significantly reduces computation time for large simulations.

In [4]:
# Import the module
base_path = '/home/selvani/MAP/pynbody/AnnaWright_startrace'

for root, dirs, files in os.walk(base_path):
    if root not in sys.path:
        print("Adding to sys.path:", root)
        sys.path.append(root)
        
import LocAtCreation_pool_rz

# Set the tangos database
# In terminal
# export TANGOS_DB_CONNECTION=/home/selvani/MAP/Data/Marvel_BN_N10.db
# os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'

Adding to sys.path: /home/selvani/MAP/pynbody/AnnaWright_startrace
Adding to sys.path: /home/selvani/MAP/pynbody/AnnaWright_startrace/Extra
Adding to sys.path: /home/selvani/MAP/pynbody/AnnaWright_startrace/RomZoomSHAnalysisScripts
Adding to sys.path: /home/selvani/MAP/pynbody/AnnaWright_startrace/RomZoomSHAnalysisScripts/__pycache__


In [5]:
import psutil
n_cpus = psutil.cpu_count(logical=True) # use up to 36 quirm cores
LocAtCreation_pool_rz.main(simpath, ss_dir, outfile_dir, n_cpus)

Stars from 42 steps left to deal with
Initializing  42
Chunks to process: [array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.001813'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000384'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.003636'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000672'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000512'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.001408'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.002432'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.002816'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.001664'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000482'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000768'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.002048'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000896'], dtype='<U41'), array(['cptmarvel.cosmo25cmb.4096g5HbwK1B

Processing snapshots:   0%|          | 0/42 [00:00<?, ?chunks/s]

Processing chunk 1/42
MyFirstStep:  001813
/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/
HaloStarsPath: /home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.001813
812 relevant stars in cptmarvel.cosmo25cmb.4096g5HbwK1BH.001813
Looking through 12871 halos
Amiga Host IDs [ 0  1  2  5 14]
FID Host IDs [-1.  1.  2.  5. 14.]
Process completed 1 snapshots, output: /home/selvani/MAP/pynbody/stellarhalo_trace_aw/cptmarvel.cosmo25cmb.4096g5HbwK1BH_stardata_001813.h5
  Completed: /home/selvani/MAP/pynbody/stellarhalo_trace_aw/cptmarvel.cosmo25cmb.4096g5HbwK1BH_stardata_001813.h5
Processing chunk 2/42
MyFirstStep:  000384
/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/
HaloStarsPath: /home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.000384
23670 relevant 

KeyError: 'No array amiga.grp for family star'

### 3) writeouthosts_rz.py

Step 3 of stellar halo pipeline                                                                                   
For each snapshot, writes out a list of halos that formed stars                                                   
between the last snapshot and this one and the number of stars formed;                                            
as in step 2, note that the IDs of these halos will be their index in                                             
the tangos database, not necessarily their amiga.grp ID. This is used                                             
to construct a unique ID for each star-forming halo in the next step.                                             
                                                                                                                  
Output: <sim>_halostarhosts.txt                                                                                   
                                                                                                                  
Usage:   python writeouthosts_rz.py <sim>                                                                         
Example: python writeouthosts_rz.py r634                                                                          
                                                                                                                  
Note that this is currently set up for MMs, but should be easily adapted                                          
by e.g., changing the paths or adding a path CL argument.    

In [ ]:
# # Import the module
# import os
# base_path = '/home/christenc/Code/python/AnnaWright_startrace'

# for root, dirs, files in os.walk(base_path):
#     if root not in sys.path:
#         sys.path.append(root)
        
import writeouthosts_rz

In [ ]:
writeouthosts_rz.main(basename, odir=outfile_dir)

#### Extra stuff charlotte added

In [ ]:
print(np.unique(tslist))
print(len(tslist))
print(len(hostlist))

In [ ]:
t = 456
tstr = str(int(t))
tmask = np.where(tslist==t)[0]
np.size(tmask)

### 4) IDUniqueHost_rz.py

Step 4 of stellar halo pipeline                                                                                                                       
Creates a unique ID for each host that forms a star. The format of                                                                                    
this ID is \<last snapshot where host was IDed\>_\<index at this snapshot\>.                                                                              
So, if a host was halo 5 at snapshot 3552 and then merged with halo 1                                                                                 
before the next snapshot, its unique ID will be 3552_5. Stars that form                                                                               
in its main progenitors will also be associated with this ID. These IDs                                                                               
are written out to a file with a similar format to <sim>_halostarhosts.txt.                                                                           
                                                                                                                                                      
Output: <sim>_uniquehalostarhosts.txt                                                                                                                 
                                                                                                                                                      
Usage:   python IDUniqueHost_rz.py <sim>                                                                                                              
Example: python IDUniqueHost_rz.py r634                                                                                                               
                                                                                                                                                      
Note that this is currently set up for MMs, but should be easily adapted                                                                              
by e.g., changing the paths or adding a path CL argument. It is also                                                                                  
designed to accommodate the phantoms that rockstar generates when it                                                                                  
temporarily loses track of a halo, which slows it down quite a bit.                                                                                   
If you're only ever going to be using it with other types of merger                                                                                   
trees, it can be simplified.  

In [7]:
# Import the module
import os
base_path = '/home/christenc/Code/python/AnnaWrite_startrace'

for root, dirs, files in os.walk(base_path):
    if root not in sys.path:
        sys.path.append(root)
        
import IDUniqueHost_rz

# Set the tangos database
# In terminal
# export TANGOS_DB_CONNECTION=/home/christenc/Storage/tangos_db/JL_r200.db
os.environ['TANGOS_DB_CONNECTION'] = '/home/christenc/Storage/tangos_db/JL_r200.db'

In [ ]:

sim = db.get_simulation(cursim+'%')

d = collections.defaultdict(list)

hsfile = odir+cursim+'_halostarhosts.txt'
ofile = odir+cursim+'_uniquehalostarhosts.txt'

main(sim, d, hsfile, ofile)


### 5) StoreUniqueHostID_rz.py

Step 5 of stellar halo pipeline
Stores the unique ID of each star particle's host at formation time.
Creates an hdf5 file that contains this in addition to all of the data
from the <sim>_stardata_<snapshot>.h5 files. Note that all star particles
that don't have a host in the snapshot after they formed will be assigned 
a unique ID of <snapshot_index>_0 and a particle host (i.e., host at
formation time) of -1. It is recommended that you use the TrackDownStars
Jupyter notebook to try to manually identify hosts for these stars and then
use FixHostIDs_rz.py to amend <sim>_allhalostardata.h5.

Output: <sim>_allhalostardata.h5

Usage:   python StoreUniqueHostID_rz.py <sim>
Example: python StoreUniqueHostID_rz.py r634 

Note that this is currently set up for MMs, but should be easily adapted 
by e.g., changing the paths or adding a path CL argument.



### 6a) TrackDownStars_rz.ipnb

### 6b) FixHostID_rz

Optional: Step 6b of stellar halo pipeline
Updates the host_ID values stored in the allhalostardata hdf5 file based
on user input. This is designed as a follow-up to TrackDownStars and can 
be used in a couple of ways. If ffile=True, this script will look for 
numpy files with names <sim>_new_ID_?.npy and will assign the particles
with the iords in a given file to new_ID. If ffile=False, it will assign
all particles with a host_ID in the old_ID list to new_ID.

Output: <sim>_allhalostardata_upd.h5

Usage:   python FixHostIDs_rz.py <sim>
Example: python FixHostIDs_rz.py r634 

The script will print out all host_IDs for which the number of assigned particles 
changed and how many particles each gained/lost. If the output looks correct,
the user should manually rename <sim>_allhalostardata_upd.h5 to <sim>_allhalostardata.h5.
It's often necessary to go back and forth between this and TrackDownStars, in which 
case I usually move the *.npy files that have already been processed to a subfolder. 


### 6c) CompTwoHalos_rz

Optional: Step 6c of stellar halo pipeline
Compares two halos to see how likely it is that one is
the main progenitor of the other based on how many particles
they have in common. This is particularly useful when you've
used a merger tree constructor that doesn't use phantoms or
some equivalent and may therefore fail to connect a halo at
snapshot1 to the same halo at snapshot3 if it lost track of it
at snapshot2. This information can then be used with FixHostIDs_rz
to merge two unique IDs and/or to create a new link in the 
relevant tangos db.

Usage: python CompTwoHalos_rz.py <sim> <halo1> <halo2>
Example: python CompTwoHalos_rz.py r718 0136_4 0192_3

Output: prints out the fraction of <halo1>'s DM particles
that are in <halo2> and vice-versa.

It takes three arguments: the simulation you're working with
and the tangos IDs of the two halos you want to compare, which
are formatted as <snapshot>_<IDatsnapshot>. Note that this ID
is assumed to be the tangos ID, not necessarily the amiga.grp
ID.


In [9]:
# boxsize, mass unit, vel unit, h

import pynbody

print(os.listdir('/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/'))

s = pynbody.load('/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.004096')
s.properties

['cptmarvel.cosmo25cmb.4096g5HbwK1BH.000482.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001792.igasorder', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000768', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002162', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001664.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003456.FeMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000291.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003968.igasorder', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002176.z0.741.AHF_profiles', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000512', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003245.amiga.grp', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001025.z1.999.AHF_halos', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003328.OxMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.002816.amiga.stat', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000672.amiga.gtp', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.001792.FeMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.000640.FeMassFrac', 'cptmarvel.cosmo25cmb.4096g5HbwK1BH.003584.HI', 'cptmarvel.cosmo25cmb

{'omegaM0': 0.24,
 'omegaL0': 0.76,
 'h': 0.7299490542599526,
 'boxsize': Unit("2.50e+04 kpc a"),
 'a': 1.0000000000142635,
 'time': Unit("1.40e+01 s kpc km**-1")}

In [11]:
print(s['mass'].units)
print(s['vel'].units)

2.31e+15 Msol
6.30e+02 km a s**-1


In [14]:
result = 'cosmo25cmb.4096g5HbwK1BH_stardata_002688.h5'
result.split('.')[-2][-6:]

'002688'